### RAG Pipelines -- Data Ingestion to vector DB PipeLine

In [1]:
import os 
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

d:\Internship\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdf's inside the directory  
def process_all_pdf(pdf_directory):
    """Process all pdf files in a directory """
    all_documents = []
    pdf_dir = Path(pdf_directory)

    # Find all pdf files recursively
    pdf_files = list(pdf_dir.glob("***/*.pdf"))

    print(f"find {len(pdf_files)} to process")

    for pdf_file in pdf_files:
        print(f"\n{pdf_file.name} pdf file to processing ")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f" loaded {len(documents)} pages")

        except Exception as e:
            print(f"Error:{e}")
        
    print(f"\nTotal documents loaded : {len(all_documents)}")
    return all_documents

# Process all documents in the data directory
all_pdf_documents = process_all_pdf("../data")

find 5 to process

1dec ticket.pdf pdf file to processing 
 loaded 3 pages

Akash Polkar Resume.pdf pdf file to processing 
 loaded 2 pages

CC_Questions_25-25[1].pdf pdf file to processing 
 loaded 2 pages

IMP Question Bank -ADAS.pdf pdf file to processing 
 loaded 2 pages

Mooc report.pdf pdf file to processing 
 loaded 12 pages

Total documents loaded : 21


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'Qt 4.8.7', 'creator': 'wkhtmltopdf 0.12.6', 'creationdate': '2025-12-01T07:54:51+00:00', 'title': 'redBus', 'source': '..\\data\\pdf\\1dec ticket.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1', 'source_file': '1dec ticket.pdf', 'file_type': 'pdf'}, page_content='Mahalaxmi Bus (Lokre Bandhu)\nNon A/C Seater / Sleeper (2+1)\nTravels\nRs. 420.0\n(inclusive of GST) \nTicket Price\nPune\nNigadi\nLIC Corner ,Nigadi\nLandmark: Lic Corner\n8828794402 8828794404 8828794405\n8828794402 8828794404 8828794405\nBoarding Point\nNesari\nNesari,Kolhapur\nDROPPING DATE & TIME:\n02/12/2025, 07:45 AM\nDropping Point\nPassenger Details\nSeat no\nAniket Polkar\n21Yrs, MALE\n11W\nredBus Ticket Information\nPune-Nesari on Monday, December 1, 2025\nTicket \nNumber: \nTUCU58609997\n   |   \nPNR No: \n469735\nHey Aniket Polkar,\nHurray!! You have saved Rs 20.00 on this booking\nCurrent redBus wallet balance\n₹\n 75.0\nTicket Details\nJourney \nDate and Time\n01/12/2025, 09

In [4]:
### Text spliting get into chunks 

def split_documents(documents,chunk_size = 1000,chunk_overlap = 200):
    """split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
        length_function = len,
        separators=["\n\n","\n"," ",""]
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    #show example of a chunk 
    if split_docs:
        print(f"\nExample chunk : ")
        print(f"Content : {split_docs[0].page_content[:200]}....")
        print(f"Metadata: {split_docs[0].metadata}")
    return split_docs

In [5]:
chunks = split_documents(all_pdf_documents)
print("hello ")
print(chunks)

Split 21 documents into 32 chunks

Example chunk : 
Content : Mahalaxmi Bus (Lokre Bandhu)
Non A/C Seater / Sleeper (2+1)
Travels
Rs. 420.0
(inclusive of GST) 
Ticket Price
Pune
Nigadi
LIC Corner ,Nigadi
Landmark: Lic Corner
8828794402 8828794404 8828794405
8828....
Metadata: {'producer': 'Qt 4.8.7', 'creator': 'wkhtmltopdf 0.12.6', 'creationdate': '2025-12-01T07:54:51+00:00', 'title': 'redBus', 'source': '..\\data\\pdf\\1dec ticket.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1', 'source_file': '1dec ticket.pdf', 'file_type': 'pdf'}
hello 
[Document(metadata={'producer': 'Qt 4.8.7', 'creator': 'wkhtmltopdf 0.12.6', 'creationdate': '2025-12-01T07:54:51+00:00', 'title': 'redBus', 'source': '..\\data\\pdf\\1dec ticket.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1', 'source_file': '1dec ticket.pdf', 'file_type': 'pdf'}, page_content='Mahalaxmi Bus (Lokre Bandhu)\nNon A/C Seater / Sleeper (2+1)\nTravels\nRs. 420.0\n(inclusive of GST) \nTicket Price\nPune\nNigadi\nLIC Corner ,Nig

### Embedding and vectorstoreDB 

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb 
from chromadb.config import Settings
import uuid 
from typing import List,Tuple,Dict,Any
from sklearn.metrics.pairwise import cosine_similarity


In [7]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2


d:\Internship\RAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anike\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Model loaded successfully. Embedding dimension: 384


### VectoreStore

In [9]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [10]:
chunks

[Document(metadata={'producer': 'Qt 4.8.7', 'creator': 'wkhtmltopdf 0.12.6', 'creationdate': '2025-12-01T07:54:51+00:00', 'title': 'redBus', 'source': '..\\data\\pdf\\1dec ticket.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1', 'source_file': '1dec ticket.pdf', 'file_type': 'pdf'}, page_content='Mahalaxmi Bus (Lokre Bandhu)\nNon A/C Seater / Sleeper (2+1)\nTravels\nRs. 420.0\n(inclusive of GST) \nTicket Price\nPune\nNigadi\nLIC Corner ,Nigadi\nLandmark: Lic Corner\n8828794402 8828794404 8828794405\n8828794402 8828794404 8828794405\nBoarding Point\nNesari\nNesari,Kolhapur\nDROPPING DATE & TIME:\n02/12/2025, 07:45 AM\nDropping Point\nPassenger Details\nSeat no\nAniket Polkar\n21Yrs, MALE\n11W\nredBus Ticket Information\nPune-Nesari on Monday, December 1, 2025\nTicket \nNumber: \nTUCU58609997\n   |   \nPNR No: \n469735\nHey Aniket Polkar,\nHurray!! You have saved Rs 20.00 on this booking\nCurrent redBus wallet balance\n₹\n 75.0\nTicket Details\nJourney \nDate and Time\n01/12/2025, 09

In [13]:
### convert the text to embeddings 
texts = [doc.page_content for doc in chunks]

## Generate the embeddings 
embeddings = embedding_manager.generate_embeddings(texts)

## store in the vector database 
vectorstore.add_documents(chunks,embeddings)


Generating embeddings for 32 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.57s/it]

Generated embeddings with shape: (32, 384)
Adding 32 documents to vector store...
Successfully added 32 documents to vector store
Total documents in collection: 32


### Retrival pipeline from vectorstore

In [16]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



In [17]:
rag_retriever

In [18]:
rag_retriever.retrieve("red bus time ")

Retrieving documents for query: 'red bus time '
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 45.15it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_a0f29111_5',
  'content': 'point.\n2\n. \nThe departure time mentioned on the ticket are only tentative \ntimings. However the bus will not leave the source before the \ntime that is mentioned on the\nticket.\n3\n. \nPassengers are required to furnish the \nfollowing at \nthe time of boarding the bus: \n(1) A digital copy of the e-ticket or m-ticket. \n(2) A valid identity proof \nFailing to do so, they may not be allowed to board the bus.\n4\n. \nChange of bus: In case the bus operator changes the type of bus due to some reason, redBus will refund the differential amount to the customer upon\nbeing intimated by the customers in 24 hours of the journey.\n5\n. \nAmenities for this bus as shown on redBus have \nbeen \nconfigured and provided by the \nbus provider (bus operator). These amenities will be provided\nunless \nthere are \nsome exceptions on certain days. Please note that redBus \nprovides \nthis information \nin good faith to help passengers to make an\ninformed d

In [19]:
rag_retriever.retrieve("PFIZER PVT LTD, Goa")

Retrieving documents for query: 'PFIZER PVT LTD, Goa'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 36.57it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_5b80f097_8',
  'content': '➢ PFIZER PVT LTD, Goa \nDepartment of warehouse currently working (One Year experience) \n \n• Handled raw material arrangements, dispensing, and online BMR entry. \n• Maintained calibration records and operated weighing balances. \n• Performed cleaning procedures for RLAF and vacuum equipment. \n• Managed online documentation with timely entries across systems. \n• Ensured compliance with SOPs and maintained GMP practices in the store \n• Updated new material data and rack locations in E-Track system',
  'metadata': {'file_type': 'pdf',
   'author': 'admin',
   'creator': 'Microsoft® Word 2021',
   'page_label': '1',
   'source': '..\\data\\pdf\\Akash Polkar Resume.pdf',
   'title': 'Microsoft Word - Nikhil Patil',
   'content_length': 498,
   'creationdate': '2025-12-10T22:34:29+05:30',
   'source_file': 'Akash Polkar Resume.pdf',
   'producer': 'Microsoft® Word 2021',
   'page': 0,
   'moddate': '2025-12-10T22:34:29+05:30',
   'doc_index': 8,


### Integrating vectordb context pipeline with LLM output 

In [26]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="openai/gpt-oss-safeguard-20b",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [33]:
answer=rag_simple("Virtulization architecture",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'Virtulization architecture'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 43.05it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
No relevant context found to answer the question.


### Enchance RAG Pipeline features

In [36]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("what is the destination in the redbus ticket?", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'what is the destination in the redbus ticket?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 56.21it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


Answer: The destination is not specified in the provided context.
Sources: [{'source': '1dec ticket.pdf', 'page': 2, 'score': 0.17093247175216675, 'preview': 'Need \nhelp? \nredBuddy\n \nis here for you!\n24x7 support\nQuick Resolution\nMultilingual\nCHAT \nWITH REDBUDDY\nTravel Guidelines\nPlease note it is mandatory to follow the travel guidelines of your source and \ndestination state of \ntravel.\nView Guidelines: \nhttps://bit.ly/redbus-guidelines\nTerms and conditi...'}]
Confidence: 0.17093247175216675
Context Preview: Need 
help? 
redBuddy
 
is here for you!
24x7 support
Quick Resolution
Multilingual
CHAT 
WITH REDBUDDY
Travel Guidelines
Please note it is mandatory to follow the travel guidelines of your source and 
destination state of 
travel.
View Guidelines: 
https://bit.ly/redbus-guidelines
Terms and conditi


In [37]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("Cloud computing", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'Cloud computing'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 52.50it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
1 
 
PIMPRI CHINCHWAD COLLEGE OF ENGINEERING, PUNE 
(An Autonomous Institute affiliated to Savitribai Phule Pune University) 
DEPARTMENT OF INFORMATION TECHNOLOGY 
 
 
 
NPTEL COURSE REPORT 
 
on 
“Cloud Computing” 
 
 
Submitted by: 
Name of Student:

 Aniket Vilas Polkar 
Roll No.: 122B1F104 
Class & Division: BTech-B 
 
 
Academic Year: 2025–26

2.1 Definition 
Cloud Computing is defined as a model that enables on -demand network access to a 
shared pool of configurable computing resources  (like networks, servers, storage, 
applications, and services) that can be rapidly provisioned with minimal 
management effort. 
2.2 Essential Characteristics 
1. On-Demand Self-Service – Users can access resources without human 
intervention from the provider. 
2. Broad Network Access – Services accessible from laptops, mobile, browsers. 
3. Resource Pooling – Multi-tenancy and shared infrastructure. 
4. Rapid Elasticity – Scaling up or down automatically. 
5. Measured Service – Usage-based billing and monitoring. 
2.3 Cloud Deployment Models 
• Public Cloud – Hosted by third-party vendors. (AWS, GCP, Azure) 
• Private Cloud – Exclusively used by one organization. 
• Hybrid Cloud – Combination of public and private. 
• Community Cloud – Shared